In [9]:
# # Find the latest version of spark from https://downloads.apache.org/spark/
# # https://downloads.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
# # Install Java
# !apt-get update
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null
# # Download and install Apache Spark
# # Ensure this URL points to a valid Spark tarball
# spark_version = '3.5.2'
# hadoop_version = '3'
# spark_filename = f'spark-{spark_version}-bin-hadoop{hadoop_version}'
# spark_url = f'https://downloads.apache.org/spark/spark-{spark_version}/{spark_filename}.tgz'
# !wget -q {spark_url}
# !tar xf {spark_filename}.tgz
# # Install findspark
# !pip install -q findspark
# # Set Environment Variables
# import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = f"/content/{spark_filename}"
# # Initialize findspark to find Spark
# import findspark
# findspark.init()

import findspark
findspark.init()
import time
from pyspark.sql import SparkSession
from pyspark import SparkFiles


# Stop the current Spark session
spark.stop()

# Create a new Spark session
spark = SparkSession.builder.appName("NewSparkSession").getOrCreate()

# Verify Spark version
print(spark.version)




3.5.2


In [10]:
# 1. Read in the AWS S3 bucket into a DataFrame.
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [11]:
# 2. Create a temporary view of the DataFrame.

spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")

# To verify, you can run a query:
spark.sql("SELECT * FROM home_sales LIMIT 5").show()




+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [12]:
# 3. Calculate the average price for a four bedroom house sold per year, rounded to two decimal places.
query = """
    SELECT date_format(date, 'yyyy') as year, 
           ROUND(AVG(price), 2) as avg_price
    FROM home_sales
    WHERE bedrooms = 4
    GROUP BY year
    ORDER BY year
"""
result = spark.sql(query)
result.show()



+----+---------+
|year|avg_price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [13]:

# 4. Calculate the average price of homes built each year, having 3 bedrooms and 3 bathrooms, rounded to two decimal places.
query = """
    SELECT date_built, 
           ROUND(AVG(price), 2) as avg_price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY date_built
    ORDER BY date_built
"""
result = spark.sql(query)
result.show()




+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [14]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

query = """
    SELECT date_built, 
           ROUND(AVG(price), 2) as avg_price
    FROM home_sales
    WHERE bedrooms = 3 
      AND bathrooms = 3 
      AND floors = 2
      AND sqft_living >= 2000
    GROUP BY date_built
    ORDER BY date_built
"""
result = spark.sql(query)
result.show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [15]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating. 
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()
query = """
    SELECT view, 
           ROUND(AVG(price), 2) as avg_price
    FROM home_sales
    GROUP BY view
    HAVING AVG(price) >= 350000
    ORDER BY view DESC
"""
result = spark.sql(query)
result.show()

# End the timer and calculate the runtime
end_time = time.time()
run_time = end_time - start_time

print(f"Query run time: {run_time:.4f} seconds")


+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

Query run time: 0.1801 seconds


In [16]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales")


DataFrame[]

In [17]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [18]:
# 9. Using the cached data, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the uncached runtime.

# Define the cached query execution time
start_time_cached = time.time()

# Run the cached query
query = """
    SELECT view, 
           ROUND(AVG(price), 2) as avg_price
    FROM home_sales
    GROUP BY view
    HAVING AVG(price) >= 350000
    ORDER BY view DESC
"""
result_cached = spark.sql(query)
result_cached.show()

# End the timer for the cached query
end_time_cached = time.time()

# Calculate the runtime for the cached query
run_time_cached = end_time_cached - start_time_cached

# Output the cached query runtime
print(f"Cached query run time: {run_time_cached:.4f} seconds")

# If you have the uncached



+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

Cached query run time: 0.1040 seconds


In [21]:
# Change the output path to a local directory, not managed by iCloud
output_path = "/Users/jenleemcnew/Desktop/home_sales_partitioned"

# Write the DataFrame to Parquet format, partitioned by the "date_built" field
df.write.mode("overwrite").partitionBy("date_built").parquet(output_path)




In [22]:
# 11. Read the parquet formatted data.
parquet_data = spark.read.parquet("path_to_output_directory/home_sales_partitioned")

# Show the top rows of the DataFrame to verify
parquet_data.show()


+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a2-86f3-46b...|2022-02-18|419543|       3|        2|       1642|   12826|

In [23]:
# 12. Create a temporary table for the parquet data.
parquet_data.createOrReplaceTempView("home_sales_parquet")

# Verify the table creation by running a query
spark.sql("SELECT * FROM home_sales_parquet LIMIT 5").show()


+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
+--------------------+----------+------+--------+---------+-----------+--------+

In [24]:
# 13. Using the parquet DataFrame, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the cached runtime.

# Start the timer for the Parquet DataFrame query
start_time_parquet = time.time()

# Run the query on the Parquet DataFrame
query = """
    SELECT view, 
           ROUND(AVG(price), 2) as avg_price
    FROM home_sales_parquet
    GROUP BY view
    HAVING AVG(price) >= 350000
    ORDER BY view DESC
"""
result_parquet = spark.sql(query)
result_parquet.show()

# End the timer for the Parquet query
end_time_parquet = time.time()

# Calculate the runtime for the Parquet query
run_time_parquet = end_time_parquet - start_time_parquet

# Output the Parquet query runtime
print(f"Parquet query run time: {run_time_parquet:.4f} seconds")

# If you have the cached runtime stored in `run_time_cached`, you can compare it:
print(f"Cached query run time: {run_time_cached:.4f} seconds")


+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

Parquet query run time: 0.1891 seconds
Cached query run time: 0.1040 seconds


In [25]:
# 14. Uncache the home_sales temporary table.
spark.sql("UNCACHE TABLE home_sales")

# Verify if the table is uncached
print(spark.catalog.isCached("home_sales"))  # This should return False if the table is uncached


False


In [26]:
# 15. Check if the home_sales table is no longer cached
is_cached = spark.catalog.isCached("home_sales")

# Output the result
if not is_cached:
    print("The home_sales table is no longer cached.")
else:
    print("The home_sales table is still cached.")


The home_sales table is no longer cached.
